# ONNX Utils

A collection of ONNX utils in one MLRun function. The function includes the following handlers:

1. [to_onnx](#handler1) - Convert your model into `onnx` format.
2. [optimize](#handler2) - Perform ONNX optimizations using `onnxmodeloptimizer` on a given ONNX model.

<a id="handler1"></a>

## 1. to_onnx

### 1.1. Docs
Convert the given model to an ONNX model.

#### Parameters:
* **`context`**: `mlrun.MLClientCtx` - The MLRun function execution context
* **`model_path`**: `str` - The model path store object.
* **`onnx_model_name`**: `str = None` - The name to use to log the converted ONNX model. If not given, the given `model_name` will be used with an additional suffix `_onnx`. Defaulted to None.
* **`optimize_model`**: `bool = True` - Whether to optimize the ONNX model using 'onnxoptimizer' before saving the model. Defaulted to True.
* **`framework`**: `str = None` - The model's framework. If None, it will be read from the 'framework' label of the model artifact provided. Defaulted to None.
* **`framework_kwargs`**: `Dict[str, Any] = None` - Additional arguments each framework may require in order to convert to ONNX. *To get the doc string of the desired framework onnx conversion function, **pass "help"**.*

#### Supported keyword arguments (`framework_kwargs`) per framework:
`tensorflow.keras`:
* **`input_signature`**: `List[Tuple[Tuple[int], str]] = None` - A list of the input layers shape and data type properties. Expected to receive a list where each element is an input layer tuple. An input layer tuple is a tuple of:
  * [0] = Layer's shape, a tuple of integers.
  * [1] = Layer's data type, a mlrun.data_types.ValueType string.

  If None, the input signature will be tried to be read automatically before converting to ONNX or from the model artifact if available. Defaulted to None.

`torch`:
* **`input_signature`**: `List[Tuple[Tuple[int], str]] = None` - A list of the input layers shape and data type properties. Expected to receive a list where each element is an input layer tuple. An input layer tuple is a tuple of:
  * [0] = Layer's shape, a tuple of integers.
  * [1] = Layer's data type, a mlrun.data_types.ValueType string.

  If None, the input signature will be read from the model artifact if available. Defaulted to None.
* **`input_layers_names`**: `List[str] = None` - List of names to assign to the input nodes of the graph in order. All of the other parameters (inner layers) can be set as well by passing additional names in the list. The order is by the order of the parameters in the model. If None, the inputs will be read from the handler's inputs. If its also None, it is defaulted to: "input_0", "input_1", ...
* **`output_layers_names`**: `List[str] = None` - List of names to assign to the output nodes of the graph in order. If None, the outputs will be read from the handler's outputs. If its also None, it is defaulted to: "output_0" (for multiple outputs, this parameter must be provided).
* **`param dynamic_axes`**: `Dict[str, Dict[int, str]] = None` - If part of the input / output shape is dynamic, like (batch_size, 3, 32, 32) you can specify it by giving a dynamic axis to the input / output layer by its name as follows:
```python
{
    "input layer name": {0: "batch_size"},
    "output layer name": {0: "batch_size"},
}
```
If provided, the 'is_batched' flag will be ignored. Defaulted to None.
* **`is_batched`**: `bool = True` - Whether to include a batch size as the first axis in every input and output layer. Defaulted to True. Will be ignored if 'dynamic_axes' is provided.

### 1.2. Demo

We will use the `PyTorch` framework, a `MobileNetV2` as our model and we will convert it to ONNX using the `to_onnx` handler.

1.2.1. First we will set the artifact path for our model to be saved in and choose the models names:

In [1]:
import os
import tempfile

# Set environment variables BEFORE importing mlrun:
os.environ["MLRUN_DBPATH"] = "https://mlrun-api.default-tenant.app.innovation-dev.iguazio-cd2.com"
os.environ["V3IO_USERNAME"] = "omerm"
os.environ["V3IO_ACCESS_KEY"] = "618ed0ad-6b68-4be8-8785-b1124437eae2"

# Use a temporary directory for model artifacts (safe cleanup):
ARTIFACT_PATH = tempfile.mkdtemp()
os.environ["MLRUN_ARTIFACT_PATH"] = ARTIFACT_PATH

# Project name:
PROJECT_NAME = "onnx-utils"

# Choose our model's name:
MODEL_NAME = "mobilenetv2"

# Choose our ONNX version model's name:
ONNX_MODEL_NAME = "onnx_mobilenetv2"

# Choose our optimized ONNX version model's name:
OPTIMIZED_ONNX_MODEL_NAME = "optimized_onnx_mobilenetv2"

1.2.2. Download the model from `torchvision.models` and log it with MLRun's `PyTorchModelHandler`:

In [8]:
# mlrun: start-code

In [2]:
import torchvision

import mlrun
from mlrun.frameworks.pytorch import PyTorchModelHandler


def get_model(context: mlrun.MLClientCtx, model_name: str):
    # Download the MobileNetV2 model:
    model = torchvision.models.mobilenet_v2()

    # Initialize a model handler for logging the model:
    model_handler = PyTorchModelHandler(
        model_name=model_name,
        model=model,
        model_class="mobilenet_v2",
        modules_map={"torchvision.models": "mobilenet_v2"},
        context=context,
    )

    # Log the model:
    model_handler.log()

In [10]:
# mlrun: end-code

In [3]:
import mlrun

# Create or get the MLRun project:
project = mlrun.get_or_create_project(PROJECT_NAME, context="./")

# Create the function parsing this notebook's code using 'code_to_function':
get_model_function = mlrun.code_to_function(
    name="get_mobilenetv2",
    project=PROJECT_NAME,
    kind="job",
    image="mlrun/ml-models"
)

# Run the function to log the model:
get_model_run = get_model_function.run(
    handler="get_model",
    output_path=ARTIFACT_PATH,
    params={
        "model_name": MODEL_NAME
    },
    local=True
)

> 2026-02-10 16:14:24,932 [info] Created and saved project: {"context":"./","from_template":null,"name":"onnx-utils","overwrite":false,"save":true}
> 2026-02-10 16:14:24,933 [info] Project created successfully: {"project_name":"onnx-utils","stored_in_db":true}
> 2026-02-10 16:14:31,659 [info] Storing function: {"db":null,"name":"get-mobilenetv2-get-model","uid":"7b9d1b54375b44e191d73685a382c910"}


project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results,artifact_uris
onnx-utils,...a382c910,0,Feb 10 14:14:32,NaT,completed,run,get-mobilenetv2-get-model,v3io_user=omermkind=localowner=omermhost=M-KCX16N69X3,,model_name=mobilenetv2,,mobilenetv2_modules_map.json=store://artifacts/onnx-utils/#0@7b9d1b54375b44e191d73685a382c910model=store://models/onnx-utils/mobilenetv2#0@7b9d1b54375b44e191d73685a382c910^e0393bc5b070fd55cc57cecb94160ce412498e0f


> 2026-02-10 16:14:34,427 [info] Run execution finished: {"name":"get-mobilenetv2-get-model","status":"completed"}


1.2.4. Import the `onnx_utils` MLRun function and run it:

In [4]:
# Import the ONNX function from the marketplace:
onnx_utils_function = mlrun.import_function("hub://onnx_utils", project=PROJECT_NAME)

# Construct the model path from the run directory structure:
model_path = os.path.join(ARTIFACT_PATH, "get-mobilenetv2-get-model", "0", "model")
modules_map_path = os.path.join(ARTIFACT_PATH, "get-mobilenetv2-get-model", "0", "mobilenetv2_modules_map.json.json")

# Run the function to convert our model to ONNX:
to_onnx_run = onnx_utils_function.run(
    handler="to_onnx",
    output_path=ARTIFACT_PATH,
    params={
        "model_name": MODEL_NAME,
        "model_path": model_path,
        "load_model_kwargs": {
            "model_name": MODEL_NAME,
            "model_class": "mobilenet_v2",
            "modules_map": modules_map_path,
        },
        "onnx_model_name": ONNX_MODEL_NAME,
        "optimize_model": False,  # <- For optimizing it later in the demo, we mark the flag as False
        "framework_kwargs": {"input_signature": [((32, 3, 224, 224), "float32")]},
    },
    local=True
)

> 2026-02-10 16:14:48,519 [info] Storing function: {"db":null,"name":"onnx-utils-to-onnx","uid":"95deb2c7dbf0460291efb25c48eeebd7"}


project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results,artifact_uris
onnx-utils,...48eeebd7,0,Feb 10 14:14:49,NaT,completed,run,onnx-utils-to-onnx,v3io_user=omermkind=localowner=omermhost=M-KCX16N69X3,,"model_name=mobilenetv2model_path=/var/folders/rn/q8gs952n26982d36y50w_2rw0000gp/T/tmpvs5qvbxr/get-mobilenetv2-get-model/0/modelload_model_kwargs={'model_name': 'mobilenetv2', 'model_class': 'mobilenet_v2', 'modules_map': '/var/folders/rn/q8gs952n26982d36y50w_2rw0000gp/T/tmpvs5qvbxr/get-mobilenetv2-get-model/0/mobilenetv2_modules_map.json.json'}onnx_model_name=onnx_mobilenetv2optimize_model=Falseframework_kwargs={'input_signature': [((32, 3, 224, 224), 'float32')]}",,model=store://models/onnx-utils/onnx_mobilenetv2#0@95deb2c7dbf0460291efb25c48eeebd7^03e4286da44d015cf5465d43e809a504d15f7f63


> 2026-02-10 16:14:53,862 [info] Run execution finished: {"name":"onnx-utils-to-onnx","status":"completed"}


1.2.5. Now we verify the ONNX model was created:

In [5]:
import os

onnx_model_file = os.path.join(ARTIFACT_PATH, "onnx-utils-to-onnx", "0", "model", "onnx_mobilenetv2.onnx")
assert os.path.isfile(onnx_model_file), f"ONNX model not found at {onnx_model_file}"
print(f"ONNX model created at: {onnx_model_file}")

ONNX model created at: /var/folders/rn/q8gs952n26982d36y50w_2rw0000gp/T/tmpvs5qvbxr/onnx-utils-to-onnx/0/model/onnx_mobilenetv2.onnx


<a id="handler2"></a>

## 2. optimize

### 2.1. Docs
Optimize the given ONNX model.

#### Parameters:
* **`context`**: `mlrun.MLClientCtx` - The MLRun function execution context
* **`model_path`**: `str` - The model path store object.
* **`optimizations`**: `List[str] = None` - List of possible optimizations. *To see what optimizations are available, **pass "help"**.* If None, all of the optimizations will be used. Defaulted to None.
* **`fixed_point`**: `bool = False` - Optimize the weights using fixed point. Defaulted to False.
* **`optimized_model_name`**: `str = None` - The name of the optimized model. If None, the original model will be overridden. Defaulted to None.

### 2.2. Demo

We will use our converted model from the last example and optimize it.

2.2.1. We will call now the `optimize` handler:

In [6]:
# Construct the ONNX model path from the run directory structure:
onnx_model_path = os.path.join(ARTIFACT_PATH, "onnx-utils-to-onnx", "0", "model")

onnx_utils_function.run(
    handler="optimize",
    output_path=ARTIFACT_PATH,
    params={
        "model_path": onnx_model_path,
        "handler_init_kwargs": {"model_name": ONNX_MODEL_NAME},
        "optimized_model_name": OPTIMIZED_ONNX_MODEL_NAME,
    },
    local=True
)

> 2026-02-10 16:15:00,639 [info] Storing function: {"db":null,"name":"onnx-utils-optimize","uid":"0c30d7af94814dcabde8152a1951fb5d"}


project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results,artifact_uris
onnx-utils,...1951fb5d,0,Feb 10 14:15:01,NaT,completed,run,onnx-utils-optimize,v3io_user=omermkind=localowner=omermhost=M-KCX16N69X3,,model_path=/var/folders/rn/q8gs952n26982d36y50w_2rw0000gp/T/tmpvs5qvbxr/onnx-utils-to-onnx/0/modelhandler_init_kwargs={'model_name': 'onnx_mobilenetv2'}optimized_model_name=optimized_onnx_mobilenetv2,,model=store://models/onnx-utils/optimized_onnx_mobilenetv2#0@0c30d7af94814dcabde8152a1951fb5d^599547984e83a664dc1c2708607d06731edb5ac2


> 2026-02-10 16:15:03,414 [info] Run execution finished: {"name":"onnx-utils-optimize","status":"completed"}


2.2.2. And now our model was optimized. Let us verify:

In [7]:
optimized_model_file = os.path.join(ARTIFACT_PATH, "onnx-utils-optimize", "0", "model", "optimized_onnx_mobilenetv2.onnx")
assert os.path.isfile(optimized_model_file), f"Optimized ONNX model not found at {optimized_model_file}"
print(f"Optimized ONNX model created at: {optimized_model_file}")

Optimized ONNX model created at: /var/folders/rn/q8gs952n26982d36y50w_2rw0000gp/T/tmpvs5qvbxr/onnx-utils-optimize/0/model/optimized_onnx_mobilenetv2.onnx


Lastly, run this code to clean up all generated files and directories:

In [ ]:
import shutil

# Clean up the temporary artifact directory:
if os.path.exists(ARTIFACT_PATH):
    shutil.rmtree(ARTIFACT_PATH)